In [ ]:
import pandas as pd
import os
import librosa
import librosa.display
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import normalize
import warnings
warnings.filterwarnings('ignore')
from sklearn.model_selection import train_test_split

import tensorflow as tf
from tensorflow.keras.layers import LSTM, Dense, BatchNormalization, Bidirectional, Dropout
from tensorflow.keras.utils import to_categorical
from sklearn.preprocessing import LabelEncoder
import numpy as np
import scipy

In [ ]:
def create_dataset():

    df = pd.read_csv('./ESC-50-master/meta/esc50.csv')
    
    category_to_class = {
    'dog': 'hayvan',
    'chirping_birds': 'hayvan',
    'vacuum_cleaner': 'arkaplan', 
    'thunderstorm': 'arkaplan',
    'door_wood_knock': 'arkaplan',
    'can_opening': 'arkaplan',
    'crow': 'arkaplan',
    'clapping': 'insan',
    'fireworks': 'arkaplan',
    'chainsaw': 'arac',
    'airplane': 'arac',
    'mouse_click': 'arkaplan',
    'pouring_water': 'arkaplan',
    'train': 'arac',
    'sheep': 'hayvan',
    'water_drops': 'arkaplan',
    'church_bells': 'arkaplan',
    'clock_alarm': 'arkaplan',
    'keyboard_typing': 'arkaplan',
    'wind': 'arkaplan',
    'footsteps': 'insan',
    'frog': 'arkaplan',
    'cow': 'hayvan',
    'brushing_teeth': 'arkaplan',
    'car_horn': 'arac',
    'crackling_fire': 'arkaplan',
    'helicopter': 'arac',
    'drinking_sipping': 'arkaplan',
    'rain': 'arkaplan',
    'insects': 'arkaplan',
    'laughing': 'insan',
    'hen': 'hayvan',
    'engine': 'arac',
    'breathing': 'insan',
    'crying_baby': 'insan', 
    'hand_saw': 'arac',
    'coughing': 'insan',
    'glass_breaking': 'arkaplan',
    'snoring' : 'insan',
    'toilet_flush': 'arkaplan',
    'pig': 'hayvan',
    'washing_machine': 'arac',
    'clock_tick': 'arkaplan',
    'sneezing' : 'insan',
    'rooster': 'hayvan',
    'sea_waves': 'arkaplan',
    'siren': 'arac',
    'cat': 'hayvan',
    'door_wood_creaks': 'arkaplan',
    'crickets': 'arkaplan',
    }

    raw_data = []
    df['class'] = df['category'].map(category_to_class)

    def load_dataset(file_name):
        audio, sample_rate = librosa.load(file_name, res_type='kaiser_fast', sr = None) 
        audio_16k = librosa.resample(audio, orig_sr=sample_rate, target_sr=16000)
        
        return audio_16k.tolist()

    raw_data_list = []
    for _, row in df.iterrows():

        file_name = f"./ESC-50-master/audio/{row['filename']}"

        data = load_dataset(file_name)

        raw_data_list.append([data, row['class'], row['fold']])
        
    raw_data_list = pd.DataFrame(raw_data_list, columns = ['feature', 'class', 'fold'])

    y = np.array(raw_data_list['class'].tolist())
    
    return raw_data_list, y 

In [ ]:
raw_data_list, y = create_dataset()

In [ ]:
data = np.array(raw_data_list['feature'].tolist())

In [ ]:
import matplotlib.pyplot as plt

fig = plt.figure(figsize=(8,4))
ax = fig.add_subplot(111)
ax.plot(data[500])
plt.show()
fig = plt.figure(figsize=(8,4))
ax = fig.add_subplot(111)
ax.plot(data[800])

In [ ]:
raw_data_list['class'].unique()

In [ ]:
y_cat = tf.keras.utils.to_categorical(raw_data_list['class'].map(dict(zip(raw_data_list['class'].unique(), range(4)))).tolist() , num_classes=len(np.unique(y)))
y_cat

In [ ]:
def normalize(img):
    '''
    Normalizes an array 
    (subtract mean and divide by standard deviation)
    '''
    eps = 0.001
    if np.std(img) != 0:
        img = (img - np.mean(img)) / np.std(img)
    else:
        img = (img - np.mean(img)) / eps
    return img


normalized_data = normalize(data)

In [ ]:
import matplotlib.pyplot as plt

fig = plt.figure(figsize=(8,4))
ax = fig.add_subplot(111)
ax.plot(normalized_data[500])
plt.show()
fig = plt.figure(figsize=(8,4))
ax = fig.add_subplot(111)
ax.plot(normalized_data[800])

In [ ]:
tr = np.where(raw_data_list['fold']<4)[0]
val = np.where(raw_data_list['fold']==4)[0]
test = np.where(raw_data_list['fold']==5)[0]

In [ ]:
from sklearn.model_selection import StratifiedKFold
import tensorflow as tf
    
from sklearn.metrics import *
bestacc = 0
from tensorflow.keras.models import Model
from tensorflow.keras.layers import LSTM, Dense, Input, Conv1D, AveragePooling1D, Attention, GlobalAveragePooling1D
from tensorflow.keras.optimizers import *
        



#for k in [-9, -6, -5, -4, -3, -2, -1, 0, 1, 2, 3]:
for k in [-5, -4, -3, -2, -1, 0, 1, 2, 3]:
 for df in [data, normalized_data]:
    

                y_train = y_cat[tr]
                y_val = y_cat[val]
                y_test = y_cat[test]
    
                x_train_copy = df[tr].copy()
                x_val_copy = df[val].copy()
                x_test_copy = df[test].copy()

                tf.random.set_seed(3)
                

            
                
                if k == -9:
                    
                    x_train_copy = librosa.feature.poly_features(y=x_train_copy, sr=16000)
                    x_val_copy = librosa.feature.poly_features(y=x_val_copy, sr=16000)
                    x_test_copy = librosa.feature.poly_features(y=x_test_copy, sr=16000)
                    
                       
                    x_train_copy = x_train_copy.reshape((x_train_copy.shape[0], x_train_copy.shape[2], x_train_copy.shape[1],))
                    x_val_copy = x_val_copy.reshape((x_val_copy.shape[0], x_val_copy.shape[2], x_val_copy.shape[1],))
                    x_test_copy = x_test_copy.reshape((x_test_copy.shape[0], x_test_copy.shape[2], x_test_copy.shape[1],))

                
             
                if k == -6:
                    
                    x_train_copy = librosa.feature.spectral_contrast(y=x_train_copy, sr=16000)
                    x_val_copy = librosa.feature.spectral_contrast(y=x_val_copy, sr=16000)
                    x_test_copy = librosa.feature.spectral_contrast(y=x_test_copy, sr=16000)
                    
                       
                    x_train_copy = x_train_copy.reshape((x_train_copy.shape[0], x_train_copy.shape[2], x_train_copy.shape[1],))
                    x_val_copy = x_val_copy.reshape((x_val_copy.shape[0], x_val_copy.shape[2], x_val_copy.shape[1],))
                    x_test_copy = x_test_copy.reshape((x_test_copy.shape[0], x_test_copy.shape[2], x_test_copy.shape[1],))


          
                if k == -4:
                    
                    x_train_copy = librosa.feature.spectral_centroid(y=x_train_copy, sr=16000)
                    x_val_copy = librosa.feature.spectral_centroid(y=x_val_copy, sr=16000)
                    x_test_copy = librosa.feature.spectral_centroid(y=x_test_copy, sr=16000)
                    
                       
                    x_train_copy = x_train_copy.reshape((x_train_copy.shape[0], x_train_copy.shape[2], x_train_copy.shape[1],))
                    x_val_copy = x_val_copy.reshape((x_val_copy.shape[0], x_val_copy.shape[2], x_val_copy.shape[1],))
                    x_test_copy = x_test_copy.reshape((x_test_copy.shape[0], x_test_copy.shape[2], x_test_copy.shape[1],))


                if k == -3:
                    
                    x_train_copy = librosa.feature.chroma_cens(y=x_train_copy, sr=16000, power=2)
                    x_val_copy = librosa.feature.chroma_cens(y=x_val_copy, sr=16000, power=2)
                    x_test_copy = librosa.feature.chroma_cens(y=x_test_copy, sr=16000, power=2)
                    
                       
                    x_train_copy = x_train_copy.reshape((x_train_copy.shape[0], x_train_copy.shape[2], x_train_copy.shape[1],))
                    x_val_copy = x_val_copy.reshape((x_val_copy.shape[0], x_val_copy.shape[2], x_val_copy.shape[1],))
                    x_test_copy = x_test_copy.reshape((x_test_copy.shape[0], x_test_copy.shape[2], x_test_copy.shape[1],))


                if k == -2:
                    
                    x_train_copy = librosa.feature.chroma_cqt(y=x_train_copy, sr=16000, power=2)
                    x_val_copy = librosa.feature.chroma_cqt(y=x_val_copy, sr=16000, power=2)
                    x_test_copy = librosa.feature.chroma_cqt(y=x_test_copy, sr=16000, power=2)
                    
                       
                    x_train_copy = x_train_copy.reshape((x_train_copy.shape[0], x_train_copy.shape[2], x_train_copy.shape[1],))
                    x_val_copy = x_val_copy.reshape((x_val_copy.shape[0], x_val_copy.shape[2], x_val_copy.shape[1],))
                    x_test_copy = x_test_copy.reshape((x_test_copy.shape[0], x_test_copy.shape[2], x_test_copy.shape[1],))



                if k == -1:
                    
                    x_train_copy = librosa.feature.chroma_stft(y=x_train_copy, sr=16000, power=2)
                    x_val_copy = librosa.feature.chroma_stft(y=x_val_copy, sr=16000, power=2)
                    x_test_copy = librosa.feature.chroma_stft(y=x_test_copy, sr=16000, power=2)
                    
                       
                    x_train_copy = x_train_copy.reshape((x_train_copy.shape[0], x_train_copy.shape[2], x_train_copy.shape[1],))
                    x_val_copy = x_val_copy.reshape((x_val_copy.shape[0], x_val_copy.shape[2], x_val_copy.shape[1],))
                    x_test_copy = x_test_copy.reshape((x_test_copy.shape[0], x_test_copy.shape[2], x_test_copy.shape[1],))


                if k == 0:
                    
                    power = librosa.feature.melspectrogram(y=x_train_copy, sr=16000, power=2)
                    x_train_copy = librosa.core.power_to_db(power)
                    
                    
                    power = librosa.feature.melspectrogram(y=x_val_copy, sr=16000, power=2)
                    x_val_copy = librosa.core.power_to_db(power)
                    
                    
                    power = librosa.feature.melspectrogram(y=x_test_copy, sr=16000, power=2)
                    x_test_copy = librosa.core.power_to_db(power)
                    
                       
                    x_train_copy = x_train_copy.reshape((x_train_copy.shape[0], x_train_copy.shape[2], x_train_copy.shape[1],))
                    x_val_copy = x_val_copy.reshape((x_val_copy.shape[0], x_val_copy.shape[2], x_val_copy.shape[1],))
                    x_test_copy = x_test_copy.reshape((x_test_copy.shape[0], x_test_copy.shape[2], x_test_copy.shape[1],))


                elif k == 1:
                    
                    x_train_copy = librosa.feature.melspectrogram(y=x_train_copy, sr=16000, power=2)
                    x_val_copy = librosa.feature.melspectrogram(y=x_val_copy, sr=16000, power=2)
                    x_test_copy = librosa.feature.melspectrogram(y=x_test_copy, sr=16000, power=2)
                    
                       
                    x_train_copy = x_train_copy.reshape((x_train_copy.shape[0], x_train_copy.shape[2], x_train_copy.shape[1],))
                    x_val_copy = x_val_copy.reshape((x_val_copy.shape[0], x_val_copy.shape[2], x_val_copy.shape[1],))
                    x_test_copy = x_test_copy.reshape((x_test_copy.shape[0], x_test_copy.shape[2], x_test_copy.shape[1],))

                    
                elif k == 2:
                    
                    x_train_copy =  scipy.fftpack.dct(x_train_copy, n=512)
                    x_val_copy =  scipy.fftpack.dct(x_val_copy, n=512)
                    x_test_copy =  scipy.fftpack.dct(x_test_copy, n=512)
                    
                  
                elif k == 3:
                    
                    x_train_copy = librosa.feature.mfcc(x_train_copy, n_mfcc=64)
                    x_val_copy = librosa.feature.mfcc(x_val_copy, n_mfcc=64)
                    x_test_copy = librosa.feature.mfcc(x_test_copy, n_mfcc=64)
                    
                    x_train_copy = x_train_copy.reshape((x_train_copy.shape[0], x_train_copy.shape[2], x_train_copy.shape[1],))
                    x_val_copy = x_val_copy.reshape((x_val_copy.shape[0], x_val_copy.shape[2], x_val_copy.shape[1],))
                    x_test_copy = x_test_copy.reshape((x_test_copy.shape[0], x_test_copy.shape[2], x_test_copy.shape[1],))

                if len(x_train_copy.shape) == 2:
                    
                    x_train_copy = x_train_copy.reshape((x_train_copy.shape[0], x_train_copy.shape[1], 1))
                    x_val_copy = x_val_copy.reshape((x_val_copy.shape[0], x_val_copy.shape[1], 1))
                    x_test_copy = x_test_copy.reshape((x_test_copy.shape[0], x_test_copy.shape[1], 1))

                    
                inp = Input(shape=(x_train_copy.shape[1],x_train_copy.shape[2]))
                x = LSTM(64, return_sequences = True)(inp)
                x = Dropout(0.1)(x)
                x = LSTM(32)(x)
                x = Dropout(0.1)(x)
                x = Dense(32, activation = 'relu')(x)
                x = Dropout(0.1)(x)
                out = Dense(y_cat.shape[1],activation='softmax')(x)

                model = Model(inputs=inp, outputs=out)

                es = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True, verbose = 0)

                optimizer = tf.keras.optimizers.Adam(learning_rate = 0.001)
                model.compile(loss="categorical_crossentropy", optimizer=optimizer, metrics=["accuracy"])
                history = model.fit(x_train_copy, y_train, batch_size=32, epochs=500, validation_data=(x_val_copy, y_val), verbose =1, callbacks = [es])


                pred_train = model.predict(x_train_copy)
                pred_val = model.predict(x_val_copy)
                pred_test = model.predict(x_test_copy)

                curacc_train = accuracy_score(np.argsort(y_train,axis=1)[:,-1], np.argsort(pred_train,axis=1)[:,-1])
                curacc_val = accuracy_score(np.argsort(y_val,axis=1)[:,-1], np.argsort(pred_val,axis=1)[:,-1])
                curacc_test = accuracy_score(np.argsort(y_test,axis=1)[:,-1], np.argsort(pred_test,axis=1)[:,-1])

                model.save('my_model.h5')

                import os

                def get_size(path):
                    size = os.path.getsize(path)
                    if size < 1024:
                        return f"{size} bytes"
                    elif size < 1024*1024:
                        return f"{round(size/1024, 2)} KB"
                    elif size < 1024*1024*1024:
                        return f"{round(size/(1024*1024), 2)} MB"
                    elif size < 1024*1024*1024*1024:
                        return f"{round(size/(1024*1024*1024), 2)} GB"

                

                print('______')
                print(['Train:', curacc_train, 'Val:', curacc_val, 'Test:', curacc_test])
                #cm_val = confusion_matrix(np.argmax(y_val,axis=1), np.argmax(pred_val,axis=1))
                #print(cm_val)

                #cm_test = confusion_matrix(np.argmax(y_test,axis=1), np.argmax(pred_test,axis=1))
                #print(cm_test)
                #print([split_name, mitr, ump, l, d, bs, lr , outact, transpose, bn, 'Model Size:', get_size('my_model.h5')])
   
'''
['Train:', 0.6758333333333333, 'Val:', 0.5725, 'Test:', 0.59]
['mfcc_normalized', 0, True, 50, 32, 4, 0.001, 'softmax', False, False, 'Model Size:', '899.37 KB']
'''

In [ ]:
k

In [ ]:
x_train_copy

In [ ]:
from keras.layers import Dropout,Embedding,GlobalMaxPooling1D, MaxPooling1D, Add, Flatten


inp = Input(shape=(x_train_copy.shape[1],x_train_copy.shape[2]))
#x = Conv1D(25, kernel_size=2)(inp)
x = LSTM(l,return_sequences= True )(inp)
x = Dropout(do)(x)
x = MaxPooling1D()(x)
x = LSTM(l)(x)
x = Dropout(do)(x)
x = Dense(d,activation='relu')(x)
x = Dropout(do)(x)
x = Dense(d,activation='relu')(x)
x = Dropout(do)(x)
out = Dense(y_cat.shape[1],activation=outact)(x)
model = Model(inputs=inp, outputs=out)

model.summary()

In [ ]:
model.save('my_model.h5')

import os

def get_size(path):
    size = os.path.getsize(path)
    if size < 1024:
        return f"{size} bytes"
    elif size < 1024*1024:
        return f"{round(size/1024, 2)} KB"
    elif size < 1024*1024*1024:
        return f"{round(size/(1024*1024), 2)} MB"
    elif size < 1024*1024*1024*1024:
        return f"{round(size/(1024*1024*1024), 2)} GB"
    
get_size('my_model.h5')

In [ ]:

pred_val = model.predict(x_val)
pred_test = model.predict(x_test)

curacc_val = accuracy_score(np.argsort(y_val,axis=1)[:,-1], np.argsort(pred_val,axis=1)[:,-1])
curacc_test = accuracy_score(np.argsort(y_test,axis=1)[:,-1], np.argsort(pred_test,axis=1)[:,-1])

cm_val = confusion_matrix(np.argmax(y_val,axis=1), np.argmax(pred_val,axis=1))
print(cm_val)

cm_test = confusion_matrix(np.argmax(y_test,axis=1), np.argmax(pred_test,axis=1))
print(cm_test)
print([curacc_val, curacc_test, mtc, bs, d ,l ,NUM_FEATURES])

In [ ]:
cm_val = confusion_matrix(np.argmax(y_val,axis=1), np.argmax(pred_val,axis=1))
print(cm_val)

cm_test = confusion_matrix(np.argmax(y_test,axis=1), np.argmax(pred_test,axis=1))
print(cm_test)

In [ ]:
y_test.sum(axis=0)

In [ ]:
def plot_confusion_matrix(cm, classes,
                          normalize=False,
                          title='Confusion matrix',
                          cmap=plt.cm.Blues):
    """
    This function prints and plots the confusion matrix.
    Normalization can be applied by setting `normalize=True`.
    """
    import itertools
    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
        print("Normalized confusion matrix")
    else:
        print('Confusion matrix, without normalization')

    fig = plt.figure(figsize=(10,7))
    

    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=45)
    plt.yticks(tick_marks, classes)

    fmt = '.2f' if normalize else 'd'
    thresh = cm.max() / 2.
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, format(cm[i, j], fmt),
                 horizontalalignment="center",
                 color="white" if cm[i, j] > thresh else "black")

    plt.ylabel('True label')
    plt.xlabel('Predicted label')
    plt.tight_layout()

In [ ]:
from sklearn.metrics import confusion_matrix

class_names = dict(zip(range(len(labelencoder.classes_)),labelencoder.classes_))

y_true = np.argsort(y,axis=1)[:,-1]
y_pred = np.argsort(ypreds,axis=1)[:,-1]

y_true
y_pred

cnf_matrix = confusion_matrix([class_names[x] for x in y_true],
                 [class_names[x] for x in y_pred],
                 labels=labelencoder.classes_)

np.set_printoptions(precision=2)

# Plot non-normalized confusion matrix
plt.figure()
plot_confusion_matrix(cnf_matrix, classes=labelencoder.classes_,
                      title='Confusion matrix, without normalization')